In [1]:
import codecs
import pandas as pd
df = pd.read_json(codecs.open('data/vnexpress.json','r','utf-8'))

In [2]:
data = pd.DataFrame(df.response['docs'])

In [3]:
data

,_version_,boost,cache,content,digest,id,segment,title,tstamp,url
0,1629222562912272384,0.181217,NaN,Vòng loại U23 châu Á 2020: Lịch thi đấu trận V...,334042da3975d84678fcbd0237f452e8,https://vnexpress.net/bong-da/thai-lan-tap-nan...,20190326072745,Vòng loại U23 châu Á 2020: Lịch thi đấu trận V...,2019-03-26T00:32:34.422Z,https://vnexpress.net/bong-da/thai-lan-tap-nan...
1,1629222567317340160,0.181020,NaN,Hồ Quang Hiếu: 'Tôi làm lại cuộc đời từ quá kh...,84e2d5054abdf3fd2cc5552afd84f2a8,https://vnexpress.net/giai-tri/ho-quang-hieu-t...,20190326072745,Hồ Quang Hiếu: 'Tôi làm lại cuộc đời từ quá kh...,2019-03-26T00:27:50.776Z,https://vnexpress.net/giai-tri/ho-quang-hieu-t...
2,1629222571960434688,0.179919,NaN,Airbus thắng đơn hàng 35 tỷ USD khi Boeing gặp...,251fb87a1ec2134bb9d7a6358364540d,https://vnexpress.net/kinh-doanh/airbus-thang-...,20190326072745,Airbus thắng đơn hàng 35 tỷ USD khi Boeing gặp...,2019-03-26T00:31:58.285Z,https://vnexpress.net/kinh-doanh/airbus-thang-...
3,1629222575095676928,0.175752,NaN,Ba phụ nữ đánh ghen bằng nước mắm và ớt bột lĩ...,0b433d2d7246e8c39bdad4caf4040a7c,https://vnexpress.net/phap-luat/ba-phu-nu-danh...,20190326072745,Ba phụ nữ đánh ghen bằng nước mắm và ớt bột lĩ...,2019-03-26T00:29:44.081Z,https://vnexpress.net/phap-luat/ba-phu-nu-danh...
4,1629222576244916224,0.176224,NaN,Nhiều người liên quan 'Vũ Nhôm' đang bị điều t...,97644989a7e4540bb6e5e4aab1976291,https://vnexpress.net/phap-luat/nhieu-nguoi-li...,20190326072745,Nhiều người liên quan 'Vũ Nhôm' đang bị điều t...,2019-03-26T00:31:53.191Z,https://vnexpress.net/phap-luat/nhieu-nguoi-li...
5,1629222576343482368,0.181038,NaN,Ông Trương Duy Nhất liên quan đến Vũ Nhôm - Vn...,4e813bb3b18ac775ecdde801a45731bb,https://vnexpress.net/phap-luat/ong-truong-duy...,20190326072745,Ông Trương Duy Nhất liên quan đến Vũ Nhôm - Vn...,2019-03-26T00:28:42.445Z,https://vnexpress.net/phap-luat/ong-truong-duy...
6,1629222577443438592,0.179806,NaN,"Chán iPhone, Apple ra dịch vụ truyền hình TV P...",8949da07c1a2ba00b2427e49240d1e00,https://vnexpress.net/so-hoa/apple-ra-dich-vu-...,20190326072745,"Chán iPhone, Apple ra dịch vụ truyền hình TV P...",2019-03-26T00:31:48.053Z,https://vnexpress.net/so-hoa/apple-ra-dich-vu-...
7,1629222577454972928,0.178835,NaN,Apple tiết lộ sân khấu trước sự kiện 25/3 - Vn...,64589ad42bb6fd1ba3fa6197a31f5490,https://vnexpress.net/so-hoa/apple-tiet-lo-san...,20190326072745,Apple tiết lộ sân khấu trước sự kiện 25/3 - Vn...,2019-03-26T00:29:23.394Z,https://vnexpress.net/so-hoa/apple-tiet-lo-san...
8,1629222590773985280,0.176828,NaN,Nỗi buồn sau chiến thắng của HLV Park Hang-seo...,8ec16e38cc421881603bb2a2e8080836,https://vnexpress.net/the-thao/noi-buon-sau-ch...,20190326072745,Nỗi buồn sau chiến thắng của HLV Park Hang-seo...,2019-03-26T00:32:29.308Z,https://vnexpress.net/the-thao/noi-buon-sau-ch...
9,1629222591212290048,0.187680,NaN,Bác sĩ xin lỗi vì phát ngôn về 'thỉnh vong' ở ...,9536f2754d6cfd2da9dfd07fb568e547,https://vnexpress.net/thoi-su/bac-si-xin-loi-v...,20190326072745,Bác sĩ xin lỗi vì phát ngôn về 'thỉnh vong' ở ...,2019-03-26T00:28:11.321Z,https://vnexpress.net/thoi-su/bac-si-xin-loi-v...


In [4]:
data.shape

(11844, 10)